# 张量运算

深度神经网络学到的所有变换都可以简化为数值数据张量上的一些张量运算（tensor operation）。


## 逐元素运算

独立地应用于张量中的每个元素的运算。下面是对逐元素 relu 运算的简单实现

In [1]:
def native_relu(x):
    assert len(x.shape) == 2 # x 是一个 Numpy 的 2D 张量
    x = x.copy() # 避免覆盖输入张量
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

In [ ]:
def native_add(x, y):
    assert len(x.shape) == 2 # x 和 y 是 Numpy 的 2D 张量
    assert x.shape == y.shape

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    
    return x


## 广播

广播包含两步：

1. 向较小的张量添加轴（叫做广播轴），使其 ndim 与较大的张量相同。
2. 将较小的张量沿着新轴重复，使其形状与较大的张量相同。